In [29]:
import pandas as pd  # importa o pandas

rock_samples = pd.read_csv('data/rocksamples.csv')  # lê arquivo de dados como um DataFrame do pandas
rock_samples.head()  # imprime as cinco primeiras linhas
rock_samples.info()  # resumo das informações 

rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x : x * 0.001)  # converte os pesos das amostras das rochas de g para kg
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)  # renomeia a coluna
rock_samples.head()

missions = pd.DataFrame()  # cria um dataframe
missions['Mission'] = rock_samples['Mission'].unique()  # cria uma coluna chamada Mission com uma linha para cada missão
missions.head()
missions.info()

sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()  # soma todos os valores da coluna Weight (kg) para cada valor exclusivo da coluna Mission
missions = pd.merge(missions, sample_total_weight, on='Mission')  # mescla o DataFrame missions com a série sample_total_weight usando a coluna Mission como índice
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)

missions['Weight diff'] = missions['Sample weight (kg)'].diff()  # mostra a diferença entre a linha atual e a precedente
missions
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)  # substitui o valor NaN por 0

missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']  # cria 3 colunas para os dados do módulo lunar
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)'] # cria 3 colunas para os dados do módulo de controle
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

# cria coluna para mostrar o valor total (soma do peso dos dois módulos)
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']

# razão da amostra do peso: 
saturnVPayload = 43500  # carga do Saturn V
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload

# calcular a média de todas essas proporções em todas as missões:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)

# criar dataframe da missão Artemis com as informações sobre as três missões:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})

# estimar o peso das amostras com base nas proporções das missões da Apollo:
artemis_mission['Sample weight from total (kg)'] = artemis_mission['Total weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio

# obter a média das duas previsões:
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from payload (kg)'] + artemis_mission['Sample weight from total (kg)'])/2

# determinar o quanto resta de cada amostra trazida das missões Apollo:
rock_samples['Remaining (kg)'] = rock_samples['Weight (kg)'] * (rock_samples['Pristine (%)'] * .01)

rock_samples.describe()

# extrair informações das amostras que estão próximas de acabar:
low_samples = rock_samples.loc[(rock_samples['Weight (kg)'] >= .16) & (rock_samples['Pristine (%)'] <= 50)]
low_samples.head()

# ver quantos tipos exclusivos temos nos DataFrames low_samples e rock_samples:
low_samples.Type.unique()
rock_samples.Type.unique()

# mostrar quantos de cada tipo são considerados pouca quantidade:
low_samples.groupby('Type')['Weight (kg)'].count()

# criar novo dataframe com as rochas necessárias:
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()

# comparar o peso total do DataFrame needed_samples com o DataFrame rock_samples:
needed_samples.groupby('Type')['Weight (kg)'].sum()
rock_samples.groupby('Type')['Weight (kg)'].sum()

# adicionar as rochas Crustals ao conjunto de amostras necessárias:
needed_samples = pd.concat([needed_samples,rock_samples.loc[rock_samples['Type'] == 'Crustal']])
needed_samples.info()

# novo dataframe com as informações consolidadas:
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()

# incluir o peso total de cada tipo de rocha coletado originalmente:
needed_sample_weights = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Total weight (kg)'}, inplace=True)

# informar o tamanho estimado de cada tipo de rocha para facilitar o processo de coleta:
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Average weight (kg)'}, inplace=True)

# obter o percentual restante de cada tipo de rocha.
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of samples'].sum()
needed_samples_overview['Percentage of rocks'] = needed_samples_overview['Number of samples'] / total_rocks

# determinar o peso médio das amostras estimadas para fazer uma recomendação para o programa Artemis:
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()

# determinar quantas de cada rocha os astronautas devem coletar:
needed_samples_overview['Weight to collect'] = needed_samples_overview['Percentage of rocks'] * artemis_ave_weight
needed_samples_overview['Rocks to collect'] = needed_samples_overview['Weight to collect'] / needed_samples_overview['Average weight (kg)']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes
0.4963026819923371
0.002848764392685611
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  ---

/tmp/ipykernel_16077/1038609115.py:121: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rock_samples.sum()


ID                                                        116038245
Mission           Apollo11Apollo11Apollo11Apollo11Apollo11Apollo...
Type              SoilSoilBasaltCoreCoreSoilBrecciaSoilSoilSoilS...
Weight (kg)                                               375.03599
Pristine (%)                                              188378.95
Remaining (kg)                                           307.832448
dtype: object